In [2]:
from transformers import AutoTokenizer, AutoModel
import os
import torch
import pandas as pd
import numpy as np
import time
from tqdm import tqdm

In [ ]:
# !pip3 install --upgrade --user transformers
# !pip3 install --upgrade --user transformers tokenizers
!pip3 install --upgrade --user torchvision pillow

In [3]:
# Load GENA-LM model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("AIRI-Institute/gena-lm-bert-large-t2t")
model = AutoModel.from_pretrained("AIRI-Institute/gena-lm-bert-large-t2t", trust_remote_code=True)
model.eval()

pytorch_model.bin:   0%|          | 0.00/1.48G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.48G [00:00<?, ?B/s]

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 1024, padding_idx=3)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (pre_attention_ln): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (post_attention_ln): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
              (s

In [21]:
# Set directories
datasets_folder = '/.../.../.../.../Unsupervised_Evaluation_LLM/PaperWork/datasets/'
output_folder_path = "/.../.../.../.../Unsupervised_Evaluation_LLM/PaperWork/genalm"
os.makedirs(output_folder_path, exist_ok=True)

In [22]:
csv_files = [file for file in os.listdir(datasets_folder) if file.endswith('.csv')]
time_records = []

In [23]:
csv_files

['test_demo_coding_vs_intergenomic_seqs.csv',
 'test_human_enhancers_cohn.csv',
 'test_human_enhancers_ensembl.csv',
 'test_human_ensembl_regulatory.csv',
 'test_human_nontata_promoters.csv',
 'test_human_ocr_ensembl.csv',
 'train_demo_coding_vs_intergenomic_seqs.csv',
 'train_human_enhancers_cohn.csv',
 'train_human_enhancers_ensembl.csv',
 'train_human_ensembl_regulatory.csv',
 'train_human_nontata_promoters.csv',
 'train_human_ocr_ensembl.csv']

In [24]:
target_files = {
     ''
}

# Filter only the desired CSV files
csv_files = [file for file in os.listdir(datasets_folder)
             if file.endswith('.csv') and file in target_files]

# Initialize time_records
time_records = []

print(csv_files)

[]


In [18]:
# Embedding function
def calculate_mean_embedding(sequence):
    inputs = tokenizer(sequence, return_tensors='pt', truncation=True, padding=True)
    with torch.no_grad():
        hidden_states = model(**inputs, output_hidden_states=True).hidden_states[-1]
    embedding_mean = hidden_states.mean(dim=1).squeeze().detach().numpy()
    return embedding_mean

In [19]:
# Process each file
for csv_file in csv_files:
    file_path = os.path.join(datasets_folder, csv_file)
    df = pd.read_csv(file_path)
    print(df.shape)
    
    start_time = time.time()
    tqdm.pandas()
    df['mean_embedding'] = df['seq'].progress_apply(calculate_mean_embedding)
    time_taken = time.time() - start_time

    embedding_file_path = os.path.join(output_folder_path, os.path.splitext(csv_file)[0] + "_GENALM_Embeddings.npy")
    np.save(embedding_file_path, df['mean_embedding'].values)
    print(f"Embeddings saved to: {embedding_file_path}")
    
    time_records.append({'file_name': csv_file, 'time_taken': time_taken})

(20843, 2)


  0%|          | 0/20843 [00:00<?, ?it/s]/home/rxa615/.local/lib/python3.10/site-packages/transformers/modeling_utils.py:1614: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
100%|██████████| 20843/20843 [1:16:00<00:00,  4.57it/s]  


Embeddings saved to: /.../.../.../.../Unsupervised_Evaluation_LLM/PaperWork/genalm/train_human_enhancers_cohn_GENALM_Embeddings.npy
